This is the code of "BERT Error Detection for STPA" (BEDS) Pipeline.

This Python Notebook was created and evaluated in Google Colab Pro (A100 GPU).

This code can be found in the GitHub repository:
https://github.com/andreyokamura-unicamp/BEDS-Pipeline

The BERT models for each Pipeline step trained in this notebook can be found in
https://huggingface.co/andreyunic23


Corresponding Author: A.T. Okamura

In [1]:
# !pip install datasets
# !pip install evaluate
# !pip install sentence-transformers==3.1.1
# !pip install transformers==4.45.2

In [2]:
import time
import pandas as pd
import numpy as np
from torch.nn import functional as F
import torch
from datasets import Dataset

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import ConfusionMatrixDisplay

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer, DataCollatorWithPadding
import evaluate

from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    losses,
)

In [3]:
# Setting seed variables

random_state = 0

In [4]:
# Define input file name here:
file_name = 'stpa-dataset.csv'

# Uncomment 4 lines below if using Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# drive_path = '/content/drive/MyDrive/Colab Notebooks/'
# file_name = drive_path+file_name

df = pd.read_csv(file_name, index_col=False, delimiter=',')

# remove unnecessary columns for training
df.drop(['domain', 'year', 'title', 'source', 'slide'], inplace=True, axis=1)
print(df.head())

                                            sentence       label validity  \
0  A collision between the ACROBOTER robotic plat...  constraint    valid   
1   A non‐patient is injured or killed by radiation.        loss  invalid   
2  A nonpatient is injured or killed in the proce...        loss  invalid   
3  A pair of controlled aircraft violates minimum...      hazard    valid   
4  A person or worker is standing/working under o...      hazard  invalid   

      faults  
0      valid  
1  condition  
2  condition  
3      valid  
4    rewrite  


#Explore the dataset

In [5]:
test_df = df.copy()
print(test_df[['sentence','faults']])

                                               sentence     faults
0     A collision between the ACROBOTER robotic plat...      valid
1      A non‐patient is injured or killed by radiation.  condition
2     A nonpatient is injured or killed in the proce...  condition
3     A pair of controlled aircraft violates minimum...      valid
4     A person or worker is standing/working under o...    rewrite
...                                                 ...        ...
1079    Avoid flight in altitude below permitted limit.        not
1080            System is capable of defending threats.        not
1081                     Verify if the runway is empty.        not
1082       Sudden braking olny in emergency situations.        not
1083        Vehicle overtaking only when road is empty.        not

[1084 rows x 2 columns]


In [6]:
# Dataset overview
df.head()

# Class distribution of each target column
print(df.label.value_counts())
print('\n')
print(df.validity.value_counts())
print('\n')
print(df.faults.value_counts())

label
hazard        424
constraint    369
loss          291
Name: count, dtype: int64


validity
valid      780
invalid    304
Name: count, dtype: int64


faults
valid        780
rewrite      122
not           77
condition     60
accident      45
Name: count, dtype: int64


In [7]:
# Distribution of valid and invalid sentences for each class
out = df.groupby('label')['validity'].value_counts(normalize=False)
print(out)
print('\n')

# Length of the longest sentence in the dataset
print(df.sentence.str.len().max())

label       validity
constraint  valid       301
            invalid      68
hazard      valid       287
            invalid     137
loss        valid       192
            invalid      99
Name: count, dtype: int64


270


In [8]:
# Generate list of valid examples for sentence similarity calculation (for use in execution example)

valid_loss_reference = df[df.label.isin(['loss']) & df.validity.isin(['valid'])]['sentence'].drop_duplicates()
valid_hazard_reference = df[df.label.isin(['hazard']) & df.validity.isin(['valid'])]['sentence'].drop_duplicates()
valid_constraint_reference = df[df.label.isin(['constraint']) & df.validity.isin(['valid'])]['sentence'].drop_duplicates()

valid_loss_reference.to_csv('valid_loss_reference.csv',index=False, header=False, sep=',')
valid_hazard_reference.to_csv('valid_hazard_reference.csv',index=False, header=False, sep=',')
valid_constraint_reference.to_csv('valid_constraint_reference.csv',index=False, header=False, sep=',')

In [9]:
step1_labels = ['loss', 'hazard', 'constraint']
step2_labels = ['valid', 'invalid']
step3_labels = ['rewrite', 'not', 'condition', 'accident', 'valid']


# Convert target labels to int as input for BERT prediction target
def label_to_int(df, name, labels):
  aux_df = df.copy()
  for i in range(len(aux_df[name])):
    aux_df.loc[i,name] = labels.index(aux_df.loc[i,name])
  return aux_df

In [10]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
  acc = accuracy_score(labels, preds)
  return {
  'accuracy': acc,
  'f1': f1,
  'precision': precision,
  'recall': recall
  }

# Path to save model
path = '/content/drive/MyDrive/Colab Notebooks/models/BEDS5/'
model_name = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [11]:
# Rename target column, convert label to int and convert to Dataset class
def format_dataset(df):
  df.rename(columns={df.columns[0]:'sentence', df.columns[1]:'labels'}, inplace=True)
  return Dataset.from_pandas(df)

# Transform input into embedding
def tokenize_function(examples):
    return tokenizer(examples['sentence'], truncation=True, padding='max_length', max_length=512, return_tensors='pt')


def train_model(train_dataset, test_dataset, num_labels):

  train_dataset = format_dataset(train_dataset)
  test_dataset = format_dataset(test_dataset)

  train_dataset = train_dataset.map(tokenize_function, batched=True)
  test_dataset = test_dataset.map(tokenize_function, batched=True)

  model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=num_labels)

  training_args = TrainingArguments(
      output_dir="beds_model",
      learning_rate=2e-5,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      num_train_epochs=12,
      weight_decay=0.01,
      eval_strategy="epoch",
      save_strategy="epoch",
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
  )

  trainer.train()


  eval = trainer.evaluate()

  return model, eval

In [12]:
aux_df = df.copy()
aux_df.reset_index(drop=True, inplace=True)

aux_df = label_to_int(aux_df, 'label', step1_labels)      # Convert Step 1 Labels
aux_df = label_to_int(aux_df, 'validity', step2_labels) # Convert Step 2 Labels
aux_df = label_to_int(aux_df, 'faults', step3_labels)      # Convert Step 3 Labels


In [13]:
# Pipeline fine-tuning and evaluation

# Create lists to save models and evaluation results obtained in each fold
model_list_step1 = []
model_list_step2_loss = []
model_list_step2_hazard = []
model_list_step2_constraint = []
model_list_step3_loss = []
model_list_step3_hazard = []
model_list_step3_constraint = []

eval_list_step1 = []
eval_list_step2_loss = []
eval_list_step2_hazard = []
eval_list_step2_constraint = []
eval_list_step3_loss = []
eval_list_step3_hazard = []
eval_list_step3_constraint = []

time_list_step1 = []
time_list_step2_loss = []
time_list_step2_hazard = []
time_list_step2_constraint = []
time_list_step3_loss = []
time_list_step3_hazard = []
time_list_step3_constraint = []


skf = StratifiedKFold(n_splits=5, random_state=random_state, shuffle=True)

for i, (train_index, val_index) in enumerate(skf.split(aux_df['sentence'].to_list(), aux_df['label'].to_list())):
    train_df = aux_df.iloc[train_index]
    val_df = aux_df.iloc[val_index]

    print('# Fold: ', i+1)

    # Step 1
    print('# Step 1')
    start = time.perf_counter()
    model_step, eval_step = train_model(train_df[['sentence','label']], val_df[['sentence','label']], 3)
    end = time.perf_counter()
    time_list_step1.append(end-start)
    model_list_step1.append(model_step)
    eval_list_step1.append(eval_step)


    # Step 2 - Loss
    print('# Step 2 - Loss')
    train_df_loss = train_df[train_df.label.isin([0])] # 'loss' == 0
    val_df_loss = val_df[val_df.label.isin([0])] # 'loss' == 0
    start = time.perf_counter()
    model_step, eval_step = train_model(train_df_loss[['sentence','validity']], val_df_loss[['sentence','validity']], 2)
    end = time.perf_counter()
    time_list_step2_loss.append(end-start)
    model_list_step2_loss.append(model_step)
    eval_list_step2_loss.append(eval_step)

    # Step 2 - Hazard
    print('# Step 2 - Hazard')
    train_df_hazard = train_df[train_df.label.isin([1])] # 'hazard' == 1
    val_df_hazard = val_df[val_df.label.isin([1])] # 'hazard' == 1
    start = time.perf_counter()
    model_step, eval_step = train_model(train_df_hazard[['sentence','validity']], val_df_hazard[['sentence','validity']], 2)
    end = time.perf_counter()
    time_list_step2_hazard.append(end-start)
    model_list_step2_hazard.append(model_step)
    eval_list_step2_hazard.append(eval_step)

    # Step 2 - Constraint
    print('# Step 2 - Constraint')
    train_df_constraint = train_df[train_df.label.isin([2])] # 'constraint' == 2
    val_df_constraint = val_df[val_df.label.isin([2])] # 'constraint' == 2
    start = time.perf_counter()
    model_step, eval_step = train_model(train_df_constraint[['sentence','validity']], val_df_constraint[['sentence','validity']], 2)
    end = time.perf_counter()
    time_list_step2_constraint.append(end-start)
    model_list_step2_constraint.append(model_step)
    eval_list_step2_constraint.append(eval_step)


    # Step 3 - Loss
    print('# Step 3 - Loss')
    train_df_loss = train_df[train_df.label.isin([0]) & train_df.validity.isin([1])] # 'loss' == 0; 'incorrect' == 1
    val_df_loss = val_df[val_df.label.isin([0]) & val_df.validity.isin([1])] # 'loss' == 0; 'incorrect' == 1
    start = time.perf_counter()
    model_step, eval_step = train_model(train_df_loss[['sentence','faults']], val_df_loss[['sentence','faults']], 4)
    end = time.perf_counter()
    time_list_step3_loss.append(end-start)
    model_list_step3_loss.append(model_step)
    eval_list_step3_loss.append(eval_step)

    # Step 3 - Hazard
    print('# Step 3 - Hazard')
    train_df_hazard = train_df[train_df.label.isin([1]) & train_df.validity.isin([1])] # 'hazard' == 1; 'incorrect' == 1
    val_df_hazard = val_df[val_df.label.isin([1]) & val_df.validity.isin([1])] # 'hazard' == 1; 'incorrect' == 1
    start = time.perf_counter()
    model_step, eval_step = train_model(train_df_hazard[['sentence','faults']], val_df_hazard[['sentence','faults']], 4)
    end = time.perf_counter()
    time_list_step3_hazard.append(end-start)
    model_list_step3_hazard.append(model_step)
    eval_list_step3_hazard.append(eval_step)

    # Step 3 - Constraint
    print('# Step3 - Constraint')
    train_df_constraint = train_df[train_df.label.isin([2]) & train_df.validity.isin([1])] # 'constraint' == 2; 'incorrect' == 1
    val_df_constraint = val_df[val_df.label.isin([2]) & val_df.validity.isin([1])] # 'constraint' == 2; 'incorrect' == 1
    start = time.perf_counter()
    model_step, eval_step = train_model(train_df_constraint[['sentence','faults']], val_df_constraint[['sentence','faults']], 2)
    end = time.perf_counter()
    time_list_step3_constraint.append(end-start)
    model_list_step3_constraint.append(model_step)
    eval_list_step3_constraint.append(eval_step)

# Fold:  1
# Step 1


/tmp/ipython-input-11-3372909156.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={df.columns[0]:'sentence', df.columns[1]:'labels'}, inplace=True)


Map:   0%|          | 0/867 [00:00<?, ? examples/s]

Map:   0%|          | 0/217 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/tmp/ipython-input-11-3372909156.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={df.columns[0]:'sentence', df.columns[1]:'labels'}, inplace=True)


Map:   0%|          | 0/55 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.597790,0.769231,0.763636,0.785714,0.833333
2,No log,0.458918,0.846154,0.783333,0.909091,0.750000
3,No log,0.389502,0.846154,0.783333,0.909091,0.750000
4,No log,0.302290,1.000000,1.000000,1.000000,1.000000
5,No log,0.243889,1.000000,1.000000,1.000000,1.000000
6,No log,0.202299,1.000000,1.000000,1.000000,1.000000
7,No log,0.170406,1.000000,1.000000,1.000000,1.000000
8,No log,0.147147,1.000000,1.000000,1.000000,1.000000
9,No log,0.127741,1.000000,1.000000,1.000000,1.000000
10,No log,0.115551,1.000000,1.000000,1.000000,1.000000


In [14]:
def get_eval_index(eval_list, time_list):
  accuracy_list = []
  precision_list = []
  recall_list = []
  f1_list = []
  score_sum = []
  for eval in eval_list:
    print(eval)
    accuracy_list.append(eval['eval_accuracy'])
    precision_list.append(eval['eval_precision'])
    recall_list.append(eval['eval_recall'])
    f1_list.append(eval['eval_f1'])
    score_sum.append(eval['eval_accuracy']+eval['eval_precision']+eval['eval_recall']+eval['eval_f1'])
  print('###### Average: ######')
  print('Accuracy:   ', np.mean(accuracy_list))
  print('Precision:  ', np.mean(precision_list))
  print('Recall:     ', np.mean(recall_list))
  print('F1-Score:   ', np.mean(f1_list))
  print('Fine-tuning:', np.mean(time_list))

  print('Highest sum of scores index:', score_sum.index(max(score_sum)))
  return score_sum.index(max(score_sum))

In [15]:
#Step 4

model_step4 = SentenceTransformer("all-mpnet-base-v2")

sentence_list = df['sentence'].to_list()
labels_list = [0] * len(sentence_list)
df_step4 = Dataset.from_dict({
    "sentence1": sentence_list,
    "sentence2": sentence_list,
    "label": labels_list,
})

loss_step4 = losses.ContrastiveTensionLossInBatchNegatives(model=model_step4)

args_step4 = SentenceTransformerTrainingArguments(
    output_dir="beds-all-mpnet-base-v2",
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    save_strategy="epoch",

)

s_trainer = SentenceTransformerTrainer(
    model=model_step4,
    args=args_step4,
    train_dataset=df_step4,
    loss=loss_step4
)
s_trainer.train()

#model_step4.push_to_hub('beds_step4')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Step,Training Loss
500,0.065800


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=816, training_loss=0.04325545418496225, metrics={'train_runtime': 121.6586, 'train_samples_per_second': 106.922, 'train_steps_per_second': 6.707, 'total_flos': 0.0, 'train_loss': 0.04325545418496225, 'epoch': 12.0})

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
# Show training results and save best models

# Step 1

print('# Step 1')
best_index = get_eval_index(eval_list_step1, time_list_step1)
#model_list_step1[best_index].push_to_hub("beds_step1")

# Step 2

print('# Step 2 - Loss')
best_index = get_eval_index(eval_list_step2_loss, time_list_step2_loss)
#model_list_step2_loss[best_index].push_to_hub("beds_step2_loss")

print('# Step 2 - Hazard')
best_index = get_eval_index(eval_list_step2_hazard, time_list_step2_hazard)
#model_list_step2_hazard[best_index].push_to_hub("beds_step2_hazard")

print('# Step 2 - Constraint')
best_index = get_eval_index(eval_list_step2_constraint, time_list_step2_constraint)
#model_list_step2_constraint[best_index].push_to_hub("beds_step2_constraint")

# Step 3

print('# Step 3 - Loss')
best_index = get_eval_index(eval_list_step3_loss, time_list_step3_loss)
#model_list_step3_loss[best_index].push_to_hub("beds_step3_loss")

print('# Step 3 - Hazard')
best_index = get_eval_index(eval_list_step3_hazard, time_list_step3_hazard)
#model_list_step3_hazard[best_index].push_to_hub("beds_step3_hazard")

print('# Step 3 - Constraint')
best_index = get_eval_index(eval_list_step3_constraint, time_list_step3_constraint)
#model_list_step3_constraint[best_index].push_to_hub("beds_step3_constraint")

# Step 1
{'eval_loss': 0.3690735399723053, 'eval_accuracy': 0.9354838709677419, 'eval_f1': 0.9342533256967483, 'eval_precision': 0.9317428583646938, 'eval_recall': 0.9390384299106409, 'eval_runtime': 1.5844, 'eval_samples_per_second': 136.964, 'eval_steps_per_second': 8.836, 'epoch': 12.0}
{'eval_loss': 0.2094995677471161, 'eval_accuracy': 0.9585253456221198, 'eval_f1': 0.956369183829138, 'eval_precision': 0.9572283421732627, 'eval_recall': 0.9555780933062881, 'eval_runtime': 1.5675, 'eval_samples_per_second': 138.436, 'eval_steps_per_second': 8.931, 'epoch': 12.0}
{'eval_loss': 0.38085705041885376, 'eval_accuracy': 0.9447004608294931, 'eval_f1': 0.9440713607380274, 'eval_precision': 0.9587628865979382, 'eval_recall': 0.936004970487729, 'eval_runtime': 1.5508, 'eval_samples_per_second': 139.929, 'eval_steps_per_second': 9.028, 'epoch': 12.0}
{'eval_loss': 0.2705400586128235, 'eval_accuracy': 0.9447004608294931, 'eval_f1': 0.9438275029822177, 'eval_precision': 0.948851148851149, 'eval_r